In [1]:
import os

In [2]:
%pwd

'c:\\Users\\pcx\\Desktop\\Projects\\blink\\BLINK ANALYTICS\\Predictive Pipeline\\blink-prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\pcx\\Desktop\\Projects\\blink\\BLINK ANALYTICS\\Predictive Pipeline\\blink-prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from powerTimeSeries.constants import *
from powerTimeSeries.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'powerTimeSeries.utils.common'

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

NameError: name 'CONFIG_FILE_PATH' is not defined

In [8]:
import os
import urllib.request as request
import zipfile
from powerTimeSeries import logger
from powerTimeSeries.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-23 19:07:56,608: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-23 19:07:56,613: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-23 19:07:56,618: INFO: common: created directory at: artifacts]
[2023-07-23 19:07:56,622: INFO: common: created directory at: artifacts/data_ingestion]


[2023-07-23 19:08:14,760: INFO: 1170291011: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 788688
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "29ce465d21f2f7620f74059ecf96ea9e89c299f906313fa71d6801a16fadeb8a"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 21F4:1734:12FE97:1ED5DE:64BD505F
Accept-Ranges: bytes
Date: Sun, 23 Jul 2023 16:07:59 GMT
Via: 1.1 varnish
X-Served-By: cache-jnb7022-JNB
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1690128480.582551,VS0,VE389
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 9316cd856345a8926c5c1ea64b38350fbe028268
Expires: Sun, 23 Jul 2023 16:12:59 GMT
Source-Age: 0

]
